In [2]:
%reset
%matplotlib inline

import numpy as np
import scipy.io
import pywt
import matplotlib.pylab as plt
from scipy import signal
from sklearn.preprocessing import StandardScaler, normalize
import mne
import pandas as pd
from scipy.signal import lfilter, butter

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# Load Data

In [5]:
#### BCI Competition 2008 – Graz data set B
if 0:
    # - already filterd between 0.5 and 30 Hz
    # - 's':              two channel data (C3,C4)
    # - 'HDR.TRIG':       beginning of each trial
    # - 'HDR.Classlabel': can contain: 1,2,NaN     => 1 and 2: class labels, NaN: test set
    #
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

    # Parameter
    freq_s = 125    # Hz
    trial_t = 8    # sec
    #
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

    # Data path
    data_path = "/Users/leonardrychly/Dropbox/[TUM]/4. WiSe 1617/Masterarbeit/Data/eeg/Gratz_data_III/3_b/O3VR.mat"
    #data_path = "/Users/leonardrychly/Dropbox/[TUM]/4. WiSe 1617/Masterarbeit/Data/eeg/Gratz_data_III/3_b/S4b.mat"
    #data_path = "/Users/leonardrychly/Dropbox/[TUM]/4. WiSe 1617/Masterarbeit/Data/eeg/Gratz_data_III/3_b/X11b.mat"

    mat = scipy.io.loadmat(data_path)    # dict_keys(['__header__', 's', '__globals__', '__version__', 'HDR'])
    raw_data = mat['s']
    labels = mat['HDR'][0][0][33].flatten()    # HDR.Classlabel
    trials = mat['HDR'][0][0][34].flatten()    # HDR.TRIG
    
    # Remove NaN labels and trials
    if 1:
        idxs_1 = np.where(labels == 1)[0]
        idxs_2 = np.where(labels == 2)[0]
        correct_idxs = np.concatenate((idxs_1, idxs_2))
        correct_idxs = np.sort(correct_idxs)
        labels = labels[correct_idxs]
        trials = trials[correct_idxs]
        
    # Interpolate NaN values
    if 1:
        raw_data = pd.DataFrame(raw_data).interpolate().values

    print('Shapes (with NaN labels): data={}, labels={}, trials={}'.format(raw_data.shape, labels.shape, trials.shape))
    print('Non NaN-labels: ', len(np.where(labels == 1)[0]) + len(np.where(labels == 2)[0]))

    
    
### Graz data (BCI Competition II, Data set III)  
if 0:
    # Parameter
    freq_s = 128    # Hz
    trial_t = 8    # sec
    
    # Data path
    data_path = "/Users/leonardrychly/Dropbox/[TUM]/4. WiSe 1617/Masterarbeit/Data/eeg/Graz_BCI Competition_II_data_III/dataset_BCIcomp1.mat"
    
    # dict_keys(['Copyright', '__version__', 'x_test', '__header__', '__globals__', 'x_train', 'y_train'])
    mat = scipy.io.loadmat(data_path)
    raw_data = mat['x_train']
    labels = mat['y_train'].flatten()
    
    raw_data = raw_data.reshape(-1,3)
    raw_data = np.hstack((raw_data.T[0].reshape((-1,1)), raw_data.T[2].reshape(-1,1)))

    trials = np.arange(0,140*1152,1152)
    
    print("raw_data =\t{}\nlabels =\t{}\ntrials =\t{}\nfreq_s =\t{} Hz".format(raw_data.shape, labels.shape, 
                                                                       trials.shape, freq_s))
    
    
### NST Data
if 1:
    # Parameter
    trial_t = 6    # sec
    
    path = "/Users/leonardrychly/Dropbox/[TUM]/4. WiSe 1617/Masterarbeit/Data/eeg"
    
    if 0: # Emec 

        data_dir1 = "/NST-datasets/Emec-data/s1.mat"
        data_dir2 = "/NST-datasets/Emec-data/s2.mat"
        data_dir3 = "/NST-datasets/Emec-data/s3.mat"


    if 1: # Christoph 

        data_dir1 = "/NST-datasets/Christoph-data/s1.mat"
        data_dir2 = "/NST-datasets/Christoph-data/s2.mat"
        data_dir3 = "/NST-datasets/Christoph-data/s3.mat"


    if 0: # Lukas

        data_dir1 = "/NST-datasets/Lukas-data/s1.mat"
        data_dir2 = "/NST-datasets/Lukas-data/s2.mat"
        data_dir3 = "/NST-datasets/Lukas-data/s3.mat"


    if 0: # Alona

        data_dir1 = "/NST-datasets/Alona-data/s1.mat"
        data_dir2 = "/NST-datasets/Alona-data/s2.mat"
        data_dir3 = "/NST-datasets/Alona-data/s3.mat"


    if 0: # Leonard

        data_dir1 = "/NST-datasets/Leonard-data/s1.mat"
        data_dir2 = "/NST-datasets/Leonard-data/s2.mat"
        data_dir3 = "/NST-datasets/Leonard-data/s3.mat"

    mat1 = scipy.io.loadmat(path + data_dir1)    # dict_keys(['__header__', 'X', '__globals__', '__version__', 'Y'])
    mat2 = scipy.io.loadmat(path + data_dir2)  
    mat3 = scipy.io.loadmat(path + data_dir3)  
    
    # read matlab data 
    raw_data1 = mat1['X'][:,1:3]
    labels1= mat1['Y'][0]
    trials1= mat1['trial'][0]
    raw_data2 = mat2['X'][:,1:3]
    labels2= mat2['Y'][0]
    trials2= mat2['trial'][0]
    raw_data3 = mat3['X'][:,1:3]
    labels3= mat3['Y'][0]
    trials3= mat3['trial'][0]


    trials2 += raw_data1.T.shape[1]
    trials3 += raw_data1.T.shape[1] + raw_data2.T.shape[1]


    #combine matrices together 
    raw_data = np.concatenate((raw_data1, raw_data2, raw_data3))
    labels = np.concatenate((labels1,labels2,labels3))
    trials = np.concatenate((trials1, trials2, trials3))

    # Remove class 3
    c3_idxs = np.where(labels==3)[0]
    labels = np.delete(labels, c3_idxs)
    trials = np.delete(trials, c3_idxs)
    
    
    freq_s = mat1['Fs'].flatten()[0]

    
    print('Shapes (with NaN labels): data={}, labels={}, trials={}'.format(raw_data.shape, labels.shape, trials.shape))
    print('Non NaN-labels: ', len(np.where(labels == 1)[0]) + len(np.where(labels == 2)[0]))

Shapes (with NaN labels): data=(557040, 3), labels=(144,), trials=(144,)
Non NaN-labels:  144


In [6]:
np.isnan(raw_data).sum(), np.isnan(labels).sum(), np.isnan(trials).sum()

(0, 0, 0)

In [7]:
# - Create MNE Object from data- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
if 0:
    info = mne.create_info(ch_names = ['eeg_c3', 'eeg_c4'],
                           ch_types = ['eeg','eeg'],
                           sfreq = freq_s)
    mne_raw = mne.io.RawArray(data = raw_data.T,
                              info = info)
    

### Band pass filter

In [8]:
# - - - SciPy Band Pass Filter - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

if 0:
    from scipy.signal import butter, lfilter, freqz

    def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        
        y = lfilter(b, a, data)
        return y


    ### Paramerter
    lowcut = 10    # Hz
    highcut = 12    # Hz

    # Plot before filter
    plt.figure()
    plt.clf()
    plt.plot(raw_data[trials[0]+2*freq_s : trials[0]+8*freq_s], alpha=0.7)
    plt.title("Unfiltered Data")
    
    # Filter
    #filtered_ch1 = butter_bandpass_filter(raw_data[:,0], lowcut, highcut, freq_s, order=5)
    #filtered_ch2 = butter_bandpass_filter(raw_data[:,1], lowcut, highcut, freq_s, order=5)         
    #raw_data = np.hstack((filtered_ch1.reshape((-1,1)), filtered_ch2.reshape(-1,1)))
    
    # Filter
    raw_filt = np.zeros(raw_data.shape)
    for i in range(raw_data.shape[1]):    # iterate over number of channels
        raw_filt.T[i] = butter_bandpass_filter(raw_data[:,i], lowcut, highcut, freq_s, order=5)
        
    raw_data = raw_filt

    # Plot after filter
    plt.figure()
    plt.clf()
    plt.plot(raw_data[trials[0]+2*freq_s :trials[0]+8*freq_s], alpha=0.7)
    plt.title("Filtered Data (8-30Hz)")
    
    print('raw shape: ', raw_data.shape)
    
    

In [11]:
if 1:
    def notch_filter(w0, Q):
        b, a = signal.iirnotch(w0, Q)
        return b, a
    def butter_bandstop_filter(data,w0, Q):
        b, a = notch_filter(w0,Q)
        y = lfilter(b, a, data)
        return y


    f0 = 50.0  # Frequency to be removed from signal (Hz)
    Q = 30.0  # Quality factor
    w0 = f0/(freq_s/2)
    
    filtered_ch1 = butter_bandstop_filter(raw_data[:,0],w0,Q)
    
    filtered_ch2 = butter_bandstop_filter(raw_data[:,1], lowcut, highcut, freq_s, order=6)         
    raw_data1 = np.hstack((filtered_ch1.reshape((-1,1)), filtered_ch2.reshape(-1,1)))


     # Plot after filter
    plt.figure()
    plt.clf()
    plt.plot(raw_data1[trials[0]:trials[0]+1224, :], alpha=0.7)
    plt.title("Notch Filtered Data (SciPy)")

    print('raw shape: ', raw_data1.shape, 'trial shape:', trials.shape, 'labels shape:', labels.shape)

NameError: name 'lowcut' is not defined

In [ ]:
# - - - MNE Band Pass Filter and PSD - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
if 0:
    # Custom filter length
    raw_temp = mne_raw.copy()
    raw_mne_filtered = raw_temp.filter(l_freq=8, h_freq=12, 
                                      filter_length='auto', l_trans_bandwidth='auto',
                                      h_trans_bandwidth='auto', n_jobs=1, method='fir', 
                                      iir_params=None, phase='zero', fir_window='hamming')
    raw_filtered = np.hstack((raw_mne_filtered[0][0].reshape(-1,1), raw_mne_filtered[1][0].reshape(-1,1)))

    # Plot filteres signal
    plt.figure()
    plt.clf()
    plt.plot(raw_filtered[trials[0]: trials[0]+1000, :], alpha=0.7)
    plt.title("Alpha-Filtered Data (MNE)")
    
        
    # Plot PSD
    if 1:
        #fig, axes = plt.subplots(nrows=1, ncols=2)

        mne_raw.plot_psd(tmin = trials[0]/freq_s, tmax = trials[0]/freq_s + 8, 
                         fmin = 8, fmax = 12, picks=0,  color = 'red')
        mne_raw.plot_psd(tmin = trials[10]/freq_s, tmax = trials[10]/freq_s + 8, 
                         fmin = 8, fmax = 12, picks=0, color = 'blue')
        
        mne_raw.plot_psd(tmin = trials[0]/freq_s, tmax = trials[0]/freq_s + 8, 
                         fmin = 8, fmax = 12)# average = False

### Normalize  Raw EEG Data

In [ ]:
if 1:
    mean = np.nanmean(raw_data, axis=0)    # = array([ 0.07365857,  0.07829843])
    std_dev = np.nanstd(raw_data, axis=0)  # = array([ 5.06285483,  4.82875916])

    raw_data_norm = (raw_data - mean) / std_dev
    #raw_data_norm = (raw_data - mean) / np.nanmax(abs(raw_data))
    
    print("Mean =\t{:.3f}\nMin =\t{:.3f}\nMax =\t{:.3f}\nStd.Dev = {:.3f}".format(np.nanmean(raw_data_norm),
                                                                                  np.nanmin(raw_data_norm),
                                                                                  np.nanmax(raw_data_norm),
                                                                                  np.nanstd(raw_data_norm)))
    raw_data = raw_data_norm

In [ ]:
if 0:    # if TRUE:  take MNE filtered data
         # if FALSE: take SciPy filtered data
    raw_data = raw_filtered

# Extract Classes

In [ ]:
c1_idxs = np.where(labels == 1)[0]
c2_idxs = np.where(labels == 2)[0]
c1_trials = trials[c1_idxs]
c2_trials = trials[c2_idxs]

# Init arrays (#trials, length_trial)
raw_ch0_c1 = np.zeros((len(c1_idxs), freq_s*trial_t))    # (... ,3072)
raw_ch1_c1 = np.zeros((len(c1_idxs), freq_s*trial_t))
raw_ch0_c2 = np.zeros((len(c2_idxs), freq_s*trial_t))
raw_ch1_c2 = np.zeros((len(c2_idxs), freq_s*trial_t))

# Add eeg trial data to array
for i,(idx_c1, idx_c2) in enumerate(zip(c1_trials, c2_trials)):
    # class 1
    raw_ch0_c1[i,:] = raw_data.T[0][idx_c1: idx_c1 + freq_s*trial_t]
    raw_ch1_c1[i,:] = raw_data.T[1][idx_c1: idx_c1 + freq_s*trial_t]
    # class 2
    raw_ch0_c2[i,:] = raw_data.T[0][idx_c2: idx_c2 + freq_s*trial_t]
    raw_ch1_c2[i,:] = raw_data.T[1][idx_c2: idx_c2 + freq_s*trial_t]
print('Shape: ', raw_ch0_c1.shape, raw_ch1_c1.shape, raw_ch0_c2.shape, raw_ch1_c2.shape)

# Average Power

### Calc Power

In [ ]:
### Power (elementwise power)
# class 1
power_ch0_c1 = np.power(raw_ch0_c1, 2)
power_ch1_c1 = np.power(raw_ch1_c1, 2)
# class 2
power_ch0_c2 = np.power(raw_ch0_c2, 2) 
power_ch1_c2 = np.power(raw_ch1_c2, 2) 

print('Shape: ',power_ch0_c1.shape, power_ch1_c1.shape, power_ch0_c2.shape, power_ch1_c2.shape)

### Calc Mean over trials

In [ ]:
### Mean Trail: mean power of all trials of one class 
# class 1
meantrial_power_ch0_c1 = np.nanmean(power_ch0_c1, axis=0) 
meantrial_power_ch1_c1 = np.nanmean(power_ch1_c1, axis=0)
# class 2
meantrial_power_ch0_c2 = np.nanmean(power_ch0_c2, axis=0) #power_ch0.mean(0)
meantrial_power_ch1_c2 = np.nanmean(power_ch1_c2, axis=0) #power_ch1.mean(0)

print('Shape: ', meantrial_power_ch0_c1.shape, meantrial_power_ch1_c1.shape, 
                 meantrial_power_ch0_c2.shape, meantrial_power_ch1_c2.shape)

### Plot

In [ ]:
if 1:
    ### Average Power over classes
    fig1, axes1 = plt.subplots(nrows=2, ncols=1)
    fig1.tight_layout()
    # class 1
    axes1[0].set_title('Average Power of Class 1 (left hand)')
    axes1[0].plot(meantrial_power_ch0_c1, label='Channel 0 (C3)', alpha=0.7, c='red')
    axes1[0].plot(meantrial_power_ch1_c1, label='Channel 1 (C4)', alpha=0.7, c='blue')
    axes1[0].legend(loc='upper right', shadow=True)
    # class 2
    axes1[1].set_title('Average Power of Class 2 (right hand)')
    axes1[1].plot(meantrial_power_ch0_c2, label='Channel 0 (C3)', alpha=0.7, c='red')
    axes1[1].plot(meantrial_power_ch1_c2, label='Channel 1 (C4)', alpha=0.7, c='blue')
    axes1[1].legend(loc='upper right', shadow=True)


    #### Average Power over channels
    #fig2, axes2 = plt.subplots(nrows=2, ncols=1)
    #fig2.tight_layout()
    ## class 1
    #axes2[0].set_title('Average Power of Channel 1 (C3)')
    #axes2[0].plot(meantrial_power_ch0_c1, label='Class 1 (left)', alpha=0.6, c='red')
    #axes2[0].plot(meantrial_power_ch0_c2, label='Class 2 (right)', alpha=0.6, c='blue')
    #axes2[0].legend(loc='upper right', shadow=True)
    ## class 2
    #axes2[1].set_title('Average Power of Channel 2 (C4)')
    #axes2[1].plot(meantrial_power_ch1_c1, label='Class 1 (left)', alpha=0.6, c='red')
    #axes2[1].plot(meantrial_power_ch1_c2, label='Class 2 (right)', alpha=0.6, c='blue')
    #axes2[1].legend(loc='upper right', shadow=True)

# Discrete Wavlet Transform

In [ ]:
trial_t

In [ ]:
# DWT Level
L = 6

### DWT (mean coeffs)

In [ ]:
### Mean DWT Coefficients
def dwt(raw_eeg_data, L):    
    wt_coeffs = pywt.wavedec(data = raw_eeg_data, 
                             wavelet = 'db6',
                             level = L)                        # For Fs = 128 Hz:
    cAL_mean = np.nanmean(wt_coeffs[0], axis=0)                # A6:  0 Hz - 1 Hz
    cDL_mean = np.nanmean(wt_coeffs[1], axis=0)                # D6:  1 Hz - 2 Hz
    cDLm1_mean = np.nanmean(wt_coeffs[2], axis=0)              # D5:  2 Hz - 4 Hz
    cDLm2_mean = np.nanmean(wt_coeffs[3], axis=0)              # D4:  4 Hz - 8 Hz
    cDLm3_mean = np.nanmean(wt_coeffs[4], axis=0)              # D3:  8 Hz - 16 Hz
    cDLm4_mean = np.nanmean(wt_coeffs[5], axis=0)              # D2:  16 Hz - 32 Hz
    cDLm5_mean = np.nanmean(wt_coeffs[6], axis=0)              # D1:  32 Hz - 64 Hz
    return [cAL_mean, cDL_mean, cDLm1_mean, cDLm2_mean, cDLm3_mean, cDLm4_mean]

# Mean coeffs over all trials
mean_coeff_ch0_c1 = dwt(raw_ch0_c1, L)
mean_coeff_ch1_c1 = dwt(raw_ch1_c1, L)
mean_coeff_ch0_c2 = dwt(raw_ch0_c2, L)
mean_coeff_ch1_c2 = dwt(raw_ch1_c2, L)



### DWT (data means)

In [ ]:
### DWT
# Range of a trial to use for DWT in seconds
start_t = 0     # 0
end_t = 6     # trial_t

wt_coeffs_ch0_c1 = pywt.wavedec(data = raw_ch0_c1[:, start_t*freq_s:end_t*freq_s].mean(axis=0), wavelet='db6',level=L)  #raw_ch0_c1.mean(axis=0)
wt_coeffs_ch1_c1 = pywt.wavedec(data = raw_ch1_c1[:, start_t*freq_s:end_t*freq_s].mean(axis=0), wavelet='db6',level=L)
wt_coeffs_ch0_c2 = pywt.wavedec(data = raw_ch0_c2[:, start_t*freq_s:end_t*freq_s].mean(axis=0), wavelet='db6',level=L)
wt_coeffs_ch1_c2 = pywt.wavedec(data = raw_ch1_c2[:, start_t*freq_s:end_t*freq_s].mean(axis=0), wavelet='db6',level=L)
wt_coeffs_ch1_c2[0].shape

In [ ]:
AL_ch0_c1 = wt_coeffs_ch0_c1[0]
AL_ch0_c2 = wt_coeffs_ch0_c2[0]
AL_ch1_c1 = wt_coeffs_ch1_c1[0]
AL_ch1_c2 = wt_coeffs_ch1_c2[0]

DL_ch0_c1 = wt_coeffs_ch0_c1[1]
DL_ch0_c2 = wt_coeffs_ch0_c2[1]
DL_ch1_c1 = wt_coeffs_ch1_c1[1]
DL_ch1_c2 = wt_coeffs_ch1_c2[1]

DLm1_ch0_c1 = wt_coeffs_ch0_c1[2]
DLm1_ch0_c2 = wt_coeffs_ch0_c2[2]
DLm1_ch1_c1 = wt_coeffs_ch1_c1[2]
DLm1_ch1_c2 = wt_coeffs_ch1_c2[2]

DLm2_ch0_c1 = wt_coeffs_ch0_c1[3]
DLm2_ch0_c2 = wt_coeffs_ch0_c2[3]
DLm2_ch1_c1 = wt_coeffs_ch1_c1[3]
DLm2_ch1_c2 = wt_coeffs_ch1_c2[3]

DLm3_ch0_c1 = wt_coeffs_ch0_c1[4]
DLm3_ch0_c2 = wt_coeffs_ch0_c2[4]
DLm3_ch1_c1 = wt_coeffs_ch1_c1[4]
DLm3_ch1_c2 = wt_coeffs_ch1_c2[4]

DLm4_ch0_c1 = wt_coeffs_ch0_c1[5]
DLm4_ch0_c2 = wt_coeffs_ch0_c2[5]
DLm4_ch1_c1 = wt_coeffs_ch1_c1[5]
DLm4_ch1_c2 = wt_coeffs_ch1_c2[5]

#D1_ch0_c1 = wt_coeffs_ch0_c1[6]
#D1_ch0_c2 = wt_coeffs_ch0_c2[6]
#D1_ch1_c1 = wt_coeffs_ch1_c1[6]
#D1_ch1_c2 = wt_coeffs_ch1_c2[6]
#
#D3_ch0_c1.shape

#### Plot DWT Coffs

In [ ]:
### AL
# class 1
fig_1, axis_1 = plt.subplots(2,1, sharex=True)
fig_1.tight_layout()
axis_1[0].plot(AL_ch0_c1, label='C3, c1')
axis_1[0].plot(AL_ch1_c1, label='C4, c1')
axis_1[0].legend()
#axis_1[0].set_yscale('log')
axis_1[0].set_title('DWT AL Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Left)'.format(L, freq_s))
axis_1[0].set_ylabel('AL')
axis_1[0].xaxis.grid()
axis_1[1].plot(AL_ch0_c2, label='C3, c2')
axis_1[1].plot(AL_ch1_c2, label='C4, c2')
axis_1[1].legend()
#axis_1[1].set_yscale('log')
axis_1[1].set_title('DWT AL Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Right)'.format(L, freq_s))
axis_1[1].set_ylabel('AL')
axis_1[1].xaxis.grid()

### DL
# class 1
fig_1, axis_1 = plt.subplots(2,1, sharex=True)
fig_1.tight_layout()
axis_1[0].plot(DL_ch0_c1, label='C3, c1',)
axis_1[0].plot(DL_ch1_c1, label='C4, c1')
axis_1[0].legend()
#axis_1[0].set_yscale('log')
axis_1[0].set_title('DWT DL Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Left)'.format(L, freq_s))
axis_1[0].set_ylabel('DL')
axis_1[0].xaxis.grid()
axis_1[1].plot(DL_ch0_c2, label='C3, c2')
axis_1[1].plot(DL_ch1_c2, label='C4, c2')
axis_1[1].legend()
#axis_1[1].set_yscale('log')
axis_1[1].set_title('DWT DL Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Right)'.format(L, freq_s))
axis_1[1].set_ylabel('DL')
axis_1[1].xaxis.grid()

### DL-1
# class 1
fig_1, axis_1 = plt.subplots(2,1, sharex=True)
fig_1.tight_layout()
axis_1[0].plot(DLm1_ch0_c1, label='C3, c1')
axis_1[0].plot(DLm1_ch1_c1, label='C4, c1')
axis_1[0].legend()
#axis_1[0].set_yscale('log')
axis_1[0].set_title('DWT DL-1 Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Left)'.format(L, freq_s))
axis_1[0].set_ylabel('DL-1')
axis_1[0].xaxis.grid()
axis_1[1].plot(DLm1_ch0_c2, label='C3, c2')
axis_1[1].plot(DLm1_ch1_c2, label='C4, c2')
axis_1[1].legend()
#axis_1[1].set_yscale('log')
axis_1[1].set_title('DWT DL-1 Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Right)'.format(L, freq_s))
axis_1[1].set_ylabel('DL-1')
axis_1[1].xaxis.grid()

### DL-2
# class 1
fig_1, axis_1 = plt.subplots(2,1, sharex=True)
fig_1.tight_layout()
axis_1[0].plot(DLm2_ch0_c1, label='C3, c1')
axis_1[0].plot(DLm2_ch1_c1, label='C4, c1')
axis_1[0].legend()
#axis_1[0].set_yscale('log')
axis_1[0].set_title('DWT DL-2 Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Left)'.format(L, freq_s))
axis_1[0].set_ylabel('DL-2')
axis_1[0].xaxis.grid()
axis_1[1].plot(DLm2_ch0_c2, label='C3, c2')
axis_1[1].plot(DLm2_ch1_c2, label='C4, c2')
axis_1[1].legend()
#axis_1[1].set_yscale('log')
axis_1[1].set_title('DWT DL-2 Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Right)'.format(L, freq_s))
axis_1[1].set_ylabel('DL-2')
axis_1[1].xaxis.grid()

### DL-3
# class 1
fig_1, axis_1 = plt.subplots(2,1, sharex=True)
fig_1.tight_layout()
axis_1[0].plot(DLm3_ch0_c1, label='C3, c1')
axis_1[0].plot(DLm3_ch1_c1, label='C4, c1')
axis_1[0].legend()
#axis_1[0].set_yscale('log')
axis_1[0].set_title('DWT DL-3 Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Left)'.format(L, freq_s))
axis_1[0].set_ylabel('DL-3')
axis_1[0].xaxis.grid()
axis_1[1].plot(DLm3_ch0_c2, label='C3, c2')
axis_1[1].plot(DLm3_ch1_c2, label='C4, c2')
axis_1[1].legend()
#axis_1[1].set_yscale('log')
axis_1[1].set_title('DWT DL-3 Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Right)'.format(L, freq_s))
axis_1[1].set_ylabel('DL-3')
axis_1[1].xaxis.grid()

#### PSD using matplotlib.pylab.psd()

In [ ]:
#power_ch0_c1, freqs_ch0_c1 = plt.psd(x=D3_ch0_c1, Fs=freq_s, label='ch0 (C3), C1 (left)')
#power_ch0_c2, freqs_ch0_c2 = plt.psd(x=D3_ch1_c1, Fs=freq_s, label='ch1 (C4), C1 (left)')
#plt.legend(loc=4)
#plt.title('Mean data DWT-D3 (Left)')

In [ ]:
#power_ch1_c1, freqs_ch1_c1 = plt.psd(x=D3_ch0_c2, Fs=freq_s, label='ch0 (C3), C2 (right)')
#power_ch1_c2, freqs_ch1_c2 = plt.psd(x=D3_ch1_c2, Fs=freq_s, label='ch1 (C4), C2 (right)')
#plt.legend(loc=4)
#plt.title('Mean data DWT-D3 (Right)')

#### PSD using welch

In [ ]:
### PSD using mean DWT-coeffs
# AL
AL_f_ch0_c1, AL_psd_ch0_c1 = signal.welch(AL_ch0_c1, fs=freq_s/(2**L), nperseg=AL_ch0_c1.shape[0])
AL_f_ch0_c2, AL_psd_ch0_c2 = signal.welch(AL_ch0_c2, fs=freq_s/(2**L), nperseg=AL_ch0_c2.shape[0])
AL_f_ch1_c1, AL_psd_ch1_c1 = signal.welch(AL_ch1_c1, fs=freq_s/(2**L), nperseg=AL_ch1_c1.shape[0])
AL_f_ch1_c2, AL_psd_ch1_c2 = signal.welch(AL_ch1_c2, fs=freq_s/(2**L), nperseg=AL_ch1_c2.shape[0])
# DL
DL_f_ch0_c1, DL_psd_ch0_c1 = signal.welch(DL_ch0_c1, fs=freq_s/(2**L), nperseg=DL_ch0_c1.shape[0])
DL_f_ch0_c2, DL_psd_ch0_c2 = signal.welch(DL_ch0_c2, fs=freq_s/(2**L), nperseg=DL_ch0_c2.shape[0])
DL_f_ch1_c1, DL_psd_ch1_c1 = signal.welch(DL_ch1_c1, fs=freq_s/(2**L), nperseg=DL_ch1_c1.shape[0])
DL_f_ch1_c2, DL_psd_ch1_c2 = signal.welch(DL_ch1_c2, fs=freq_s/(2**L), nperseg=DL_ch1_c2.shape[0])
# DL-1
DLm1_f_ch0_c1, DLm1_psd_ch0_c1 = signal.welch(DLm1_ch0_c1, fs=freq_s/(2**(L-1)), nperseg=DLm1_ch0_c1.shape[0])
DLm1_f_ch0_c2, DLm1_psd_ch0_c2 = signal.welch(DLm1_ch0_c2, fs=freq_s/(2**(L-1)), nperseg=DLm1_ch0_c2.shape[0])
DLm1_f_ch1_c1, DLm1_psd_ch1_c1 = signal.welch(DLm1_ch1_c1, fs=freq_s/(2**(L-1)), nperseg=DLm1_ch1_c1.shape[0])
DLm1_f_ch1_c2, DLm1_psd_ch1_c2 = signal.welch(DLm1_ch1_c2, fs=freq_s/(2**(L-1)), nperseg=DLm1_ch1_c2.shape[0])
# DL-2
DLm2_f_ch0_c1, DLm2_psd_ch0_c1 = signal.welch(DLm2_ch0_c1, fs=freq_s/(2**(L-2)), nperseg=DLm2_ch0_c1.shape[0])
DLm2_f_ch0_c2, DLm2_psd_ch0_c2 = signal.welch(DLm2_ch0_c2, fs=freq_s/(2**(L-2)), nperseg=DLm2_ch0_c2.shape[0])
DLm2_f_ch1_c1, DLm2_psd_ch1_c1 = signal.welch(DLm2_ch1_c1, fs=freq_s/(2**(L-2)), nperseg=DLm2_ch1_c1.shape[0])
DLm2_f_ch1_c2, DLm2_psd_ch1_c2 = signal.welch(DLm2_ch1_c2, fs=freq_s/(2**(L-2)), nperseg=DLm2_ch1_c2.shape[0])
# DL-3
DLm3_f_ch0_c1, DLm3_psd_ch0_c1 = signal.welch(DLm3_ch0_c1, fs=freq_s/(2**(L-3)), nperseg=DLm3_ch0_c1.shape[0])
DLm3_f_ch0_c2, DLm3_psd_ch0_c2 = signal.welch(DLm3_ch0_c2, fs=freq_s/(2**(L-3)), nperseg=DLm3_ch0_c2.shape[0])
DLm3_f_ch1_c1, DLm3_psd_ch1_c1 = signal.welch(DLm3_ch1_c1, fs=freq_s/(2**(L-3)), nperseg=DLm3_ch1_c1.shape[0])
DLm3_f_ch1_c2, DLm3_psd_ch1_c2 = signal.welch(DLm3_ch1_c2, fs=freq_s/(2**(L-3)), nperseg=DLm3_ch1_c2.shape[0])
# DL-4
DLm4_f_ch0_c1, DLm4_psd_ch0_c1 = signal.welch(DLm4_ch0_c1, fs=freq_s/(2**(L-4)), nperseg=DLm4_ch0_c1.shape[0])
DLm4_f_ch0_c2, DLm4_psd_ch0_c2 = signal.welch(DLm4_ch0_c2, fs=freq_s/(2**(L-4)), nperseg=DLm4_ch0_c2.shape[0])
DLm4_f_ch1_c1, DLm4_psd_ch1_c1 = signal.welch(DLm4_ch1_c1, fs=freq_s/(2**(L-4)), nperseg=DLm4_ch1_c1.shape[0])
DLm4_f_ch1_c2, DLm4_psd_ch1_c2 = signal.welch(DLm4_ch1_c2, fs=freq_s/(2**(L-4)), nperseg=DLm4_ch1_c2.shape[0])
## DL-5
#DLm5_f_ch0_c1, DLm5_psd_ch0_c1 = signal.welch(DLm5_ch0_c1, fs=freq_s, nperseg=DLm5_ch0_c1.shape[0])
#DLm5_f_ch0_c2, DLm5_psd_ch0_c2 = signal.welch(DLm5_ch0_c2, fs=freq_s, nperseg=DLm5_ch0_c2.shape[0])
#DLm5_f_ch1_c1, DLm5_psd_ch1_c1 = signal.welch(DLm5_ch1_c1, fs=freq_s, nperseg=DLm5_ch1_c1.shape[0])
#DLm5_f_ch1_c2, DLm5_psd_ch1_c2 = signal.welch(DLm5_ch1_c2, fs=freq_s, nperseg=DLm5_ch1_c2.shape[0])

### AL
# class 1
fig_1, axis_1 = plt.subplots(2,1, sharex=True)
fig_1.tight_layout()
axis_1[0].plot(AL_f_ch0_c1, AL_psd_ch0_c1, label='C3, c1')
axis_1[0].plot(AL_f_ch1_c1, AL_psd_ch1_c1, label='C4, c1')
axis_1[0].legend()
axis_1[0].set_yscale('log')
axis_1[0].set_title('PSD of DWT AL Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Left)'.format(L, freq_s))
axis_1[0].set_ylabel('PSD [dB]')
axis_1[0].xaxis.grid()
# class 2
axis_1[1].plot(AL_f_ch0_c2, AL_psd_ch0_c2, label='C3, c2')
axis_1[1].plot(AL_f_ch1_c2, AL_psd_ch1_c2, label='C4, c2')
axis_1[1].legend()
axis_1[1].set_yscale('log')
axis_1[1].set_title('PSD of DWT AL Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Right)'.format(L, freq_s))
axis_1[1].set_ylabel('PSD [dB]')
#axis_1[1].set_xticks(np.arange(0,140,5))
axis_1[1].xaxis.grid()

### DL
# class 1
fig_2, axis_2 = plt.subplots(2,1, sharex=True)
fig_2.tight_layout()
axis_2[0].plot(DL_f_ch0_c1, DL_psd_ch0_c1, label='C3, c1')
axis_2[0].plot(DL_f_ch1_c1, DL_psd_ch1_c1, label='C4, c1')
axis_2[0].legend()
axis_2[0].set_yscale('log')
axis_2[0].set_title('PSD of DWT DL Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Left)'.format(L, freq_s))
axis_2[0].set_ylabel('PSD [dB]')
axis_2[0].xaxis.grid()
# class 2
axis_2[1].plot(DL_f_ch0_c2, DL_psd_ch0_c2, label='C3, c2')
axis_2[1].plot(DL_f_ch1_c2, DL_psd_ch1_c2, label='C4, c2')
axis_2[1].legend()
axis_2[1].set_yscale('log')
axis_2[1].set_title('PSD of DWT DL Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Right)'.format(L, freq_s))
axis_2[1].set_xlabel('Frequency [Hz]')
axis_2[1].set_ylabel('PSD [dB]')
#axis_2[1].set_xticks(np.arange(0,140,5))
axis_2[1].xaxis.grid()

### DL-1
# class 1
fig_3, axis_3 = plt.subplots(2,1, sharex=True)
fig_3.tight_layout()
axis_3[0].plot(DLm1_f_ch0_c1, DLm1_psd_ch0_c1, label='C3, c1')
axis_3[0].plot(DLm1_f_ch1_c1, DLm1_psd_ch1_c1, label='C4, c1')
axis_3[0].legend()
axis_3[0].set_yscale('log')
axis_3[0].set_title('PSD of DWT DL-1 Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Left)'.format(L, freq_s))
axis_3[0].set_ylabel('PSD [dB]')
axis_3[0].xaxis.grid()
# class 2
axis_3[1].plot(DLm1_f_ch0_c2, DLm1_psd_ch0_c2, label='C3, c2')
axis_3[1].plot(DLm1_f_ch1_c2, DLm1_psd_ch1_c2, label='C4, c2')
axis_3[1].legend()
axis_3[1].set_yscale('log')
axis_3[1].set_title('PSD of DWT DL-1 Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Right)'.format(L, freq_s))
axis_3[1].set_xlabel('Frequency [Hz]')
axis_3[1].set_ylabel('PSD [dB]')
#axis_3[1].set_xticks(np.arange(0,140,5))
axis_3[1].xaxis.grid()

### DL-2
# class 1
fig_4, axis_4 = plt.subplots(2,1, sharex=True)
fig_4.tight_layout()
axis_4[0].plot(DLm2_f_ch0_c1, DLm2_psd_ch0_c1, label='C3, c1')
axis_4[0].plot(DLm2_f_ch1_c1, DLm2_psd_ch1_c1, label='C4, c1')
axis_4[0].legend()
axis_4[0].set_yscale('log')
axis_4[0].set_title('PSD of DWT DL-2 Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Left)'.format(L, freq_s))
axis_4[0].set_ylabel('PSD [dB]')
axis_4[0].xaxis.grid()
# class 2
axis_4[1].plot(DLm2_f_ch0_c2, DLm2_psd_ch0_c2, label='C3, c2')
axis_4[1].plot(DLm2_f_ch1_c2, DLm2_psd_ch1_c2, label='C4, c2')
axis_4[1].legend()
axis_4[1].set_yscale('log')
axis_4[1].set_title('PSD of DWT DL-2 Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Right)'.format(L, freq_s))
axis_4[1].set_xlabel('Frequency [Hz]')
axis_4[1].set_ylabel('PSD [dB]')
#axis_4[1].set_xticks(np.arange(0,140,5))
axis_4[1].xaxis.grid()

In [ ]:
### DL-3
# class 1
fig_5, axis_5 = plt.subplots(2,1, sharex=True)
fig_5.tight_layout()
axis_5[0].plot(DLm3_f_ch0_c1, DLm3_psd_ch0_c1, label='C3, c1')
axis_5[0].plot(DLm3_f_ch1_c1, DLm3_psd_ch1_c1, label='C4, c1')
axis_5[0].legend()
axis_5[0].set_yscale('log')
axis_5[0].set_title('PSD of DWT DL-3 Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Left)'.format(L, freq_s))
#axis_5[0].set_xlabel('Frequency [Hz]')
axis_5[0].set_ylabel('PSD [dB]')
axis_5[0].xaxis.grid()
# class 2
axis_5[1].plot(DLm3_f_ch0_c2, DLm3_psd_ch0_c2, label='C3, c2')
axis_5[1].plot(DLm3_f_ch1_c2, DLm3_psd_ch1_c2, label='C4, c2')
axis_5[1].legend()
axis_5[1].set_yscale('log')
axis_5[1].set_title('PSD of DWT DL-3 Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Right)'.format(L, freq_s))
axis_5[1].set_xlabel('Frequency [Hz]')
axis_5[1].set_ylabel('PSD [dB]')
#axis_5[1].set_xticks(np.arange(0,140,5))
axis_5[1].xaxis.grid()

### DL-4
# class 1
fig_6, axis_6 = plt.subplots(2,1, sharex=True)
fig_6.tight_layout()
axis_6[0].plot(DLm4_f_ch0_c1, DLm4_psd_ch0_c1, label='C3, c1')
axis_6[0].plot(DLm4_f_ch1_c1, DLm4_psd_ch1_c1, label='C4, c1')
axis_6[0].legend()
axis_6[0].set_yscale('log')
axis_6[0].set_title('PSD of DWT DL-4 Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Left)'.format(L, freq_s))
#axis_6[0].set_xlabel('Frequency [Hz]')
axis_6[0].set_ylabel('PSD [dB]')
axis_6[0].xaxis.grid()
# class 2
axis_6[1].plot(DLm4_f_ch0_c2, DLm4_psd_ch0_c2, label='C3, c2')
axis_6[1].plot(DLm4_f_ch1_c2, DLm4_psd_ch1_c2, label='C4, c2')
axis_6[1].legend()
axis_6[1].set_yscale('log')
axis_6[1].set_title('PSD of DWT DL-4 Coeffs (L={}, Fs={}Hz), Mean Coeffs (Class: Right)'.format(L, freq_s))
axis_6[1].set_xlabel('Frequency [Hz]')
axis_6[1].set_ylabel('PSD [dB]')
#axis_6[1].set_xticks(np.arange(0,140,5))
axis_6[1].xaxis.grid()

#### DL-5
## class 1
#fig_7, axis_7 = plt.subplots(2,1, sharex=True)
#fig_7.tight_layout()
#axis_7[0].plot(D1_f_ch0_c1, D1_psd_ch0_c1, label='C3, c1')
#axis_7[0].plot(D1_f_ch1_c1, D1_psd_ch1_c1, label='C4, c1')
#axis_7[0].legend()
#axis_7[0].set_yscale('log')
#axis_7[0].set_title('PSD of DWT-D1-Coeffs, Mean Coeffs (Class: Left)')
#axis_7[0].set_ylabel('PSD [dB]')
#axis_7[0].xaxis.grid()
## class 2
#axis_7[1].plot(D1_f_ch0_c2, D1_psd_ch0_c2, label='C3, c2')
#axis_7[1].plot(D1_f_ch1_c2, D1_psd_ch1_c2, label='C4, c2')
#axis_7[1].legend()
#axis_7[1].set_yscale('log')
#axis_7[1].set_title('PSD of DWT-D1-Coeffs, Mean Coeffs (Class: Right)')
#axis_7[1].set_xlabel('Frequency [Hz]')
#axis_7[1].set_ylabel('PSD [dB]')
##axis_7[1].set_xticks(np.arange(0,140,5))
#axis_7[1].xaxis.grid()

In [ ]:
L

In [ ]:
A6_ch0_c1, D6_ch0_c1, D5_ch0_c1, D4_ch0_c1, D3_ch0_c1, D2_ch0_c1 = mean_coeff_ch0_c1
A6_ch1_c1, D6_ch1_c1, D5_ch1_c1, D4_ch1_c1, D3_ch1_c1, D2_ch1_c1 = mean_coeff_ch1_c1
A6_ch0_c2, D6_ch0_c2, D5_ch0_c2, D4_ch0_c2, D3_ch0_c2, D2_ch0_c2 = mean_coeff_ch0_c2
A6_ch1_c2, D6_ch1_c2, D5_ch1_c2, D4_ch1_c2, D3_ch1_c2, D2_ch1_c2 = mean_coeff_ch1_c2


### PSD
# A6
psd_A6_ch0_c1 = 20*np.log10(np.abs(np.fft.rfft(A6_ch0_c1)))
psd_A6_ch1_c1 = 20*np.log10(np.abs(np.fft.rfft(A6_ch1_c1)))
psd_A6_ch0_c2 = 20*np.log10(np.abs(np.fft.rfft(A6_ch0_c2)))
psd_A6_ch1_c2 = 20*np.log10(np.abs(np.fft.rfft(A6_ch1_c2)))
f_A6 = np.linspace(0, freq_s/(2**(6)), len(psd_A6_ch0_c1))

# D4: alpha
psd_D4_ch0_c1 = 20*np.log10(np.abs(np.fft.rfft(D4_ch0_c1)))
psd_D4_ch1_c1 = 20*np.log10(np.abs(np.fft.rfft(D4_ch1_c1)))
psd_D4_ch0_c2 = 20*np.log10(np.abs(np.fft.rfft(D4_ch0_c2)))
psd_D4_ch1_c2 = 20*np.log10(np.abs(np.fft.rfft(D4_ch1_c2)))
f_D4 = np.linspace(0, freq_s/(2**(4)), len(psd_D4_ch0_c1))

# D3: beta
psd_D3_ch0_c1 = 20*np.log10(np.abs(np.fft.rfft(D3_ch0_c1)))
psd_D3_ch1_c1 = 20*np.log10(np.abs(np.fft.rfft(D3_ch1_c1)))
psd_D3_ch0_c2 = 20*np.log10(np.abs(np.fft.rfft(D3_ch0_c2)))
psd_D3_ch1_c2 = 20*np.log10(np.abs(np.fft.rfft(D3_ch1_c2)))
f_D3 = np.linspace(0, freq_s/(2**(3)), len(psd_D3_ch0_c1))


### Plot

# class 1
fig, axis = plt.subplots(2,1, sharex=True)
fig.tight_layout()
axis[0].plot(f_A6, psd_A6_ch0_c1, label='C3, c1')
axis[0].plot(f_A6, psd_A6_ch1_c1, label='C4, c1')
axis[0].legend()
axis[0].set_title('PSD of DWT A6, Mean Data (Class: Left)'.format(L, freq_s))
axis[0].set_ylabel('PSD [dB]')
axis[0].xaxis.grid()
# class 2
axis[1].plot(f_A6, psd_A6_ch0_c2, label='C3, c2')
axis[1].plot(f_A6, psd_A6_ch1_c2, label='C4, c2')
axis[1].legend()
axis[1].set_title('PSD of DWT A6, Mean Data (Class: Right)'.format(L, freq_s))
axis[1].set_xlabel('Frequency [Hz]')
axis[1].set_ylabel('PSD [dB]')
axis[1].xaxis.grid()

# class 1
fig, axis = plt.subplots(2,1, sharex=True)
fig.tight_layout()
axis[0].plot(f_D4, psd_D4_ch0_c1, label='C3, c1')
axis[0].plot(f_D4, psd_D4_ch1_c1, label='C4, c1')
axis[0].legend()
axis[0].set_title('PSD of DWT D4, Mean Data (Class: Left)'.format(L, freq_s))
axis[0].set_ylabel('PSD [dB]')
axis[0].xaxis.grid()
# class 2
axis[1].plot(f_D4, psd_D4_ch0_c2, label='C3, c2')
axis[1].plot(f_D4, psd_D4_ch1_c2, label='C4, c2')
axis[1].legend()
axis[1].set_title('PSD of DWT D4, Mean Data (Class: Right)'.format(L, freq_s))
axis[1].set_xlabel('Frequency [Hz]')
axis[1].set_ylabel('PSD [dB]')
axis[1].xaxis.grid()

# class 1
fig, axis = plt.subplots(2,1, sharex=True)
fig.tight_layout()
axis[0].plot(f_D3, psd_D3_ch0_c1, label='C3, c1')
axis[0].plot(f_D3, psd_D3_ch1_c1, label='C4, c1')
axis[0].legend()
axis[0].set_title('PSD of DWT D3, Mean Data (Class: Left)'.format(L, freq_s))
axis[0].set_ylabel('PSD [dB]')
axis[0].xaxis.grid()
# class 2
axis[1].plot(f_D3, psd_D3_ch0_c2, label='C3, c2')
axis[1].plot(f_D3, psd_D3_ch1_c2, label='C4, c2')
axis[1].legend()
axis[1].set_title('PSD of DWT D3, Mean Data (Class: Right)'.format(L, freq_s))
axis[1].set_xlabel('Frequency [Hz]')
axis[1].set_ylabel('PSD [dB]')
axis[1].xaxis.grid()